In [ ]:
# Utils packages
import math
from datetime import datetime
from pathlib import Path, PurePosixPath

# Visualization packages
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from PIL import Image

# Analysis packages
import seaborn as sns
import pandas as pd

# Model packages
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.experimental import numpy as np

## Read dataset file

In [ ]:
DATA_PATH = "data" # Directory to drop collected data.

In [ ]:
hentais_df = pd.read_json(
    Path(DATA_PATH) / "preprocessed" / "metadata.ndjson",
    orient="records",
    lines=True,
    encoding="utf8"
)
hentais_df = hentais_df.assign(
    filename=hentais_df["filename"].apply(
        lambda filename: [
            str(PurePosixPath(DATA_PATH) / "preprocessed" / Path(filename))
            for filename in filename
        ]
    )
)
hentais_df

## Data preparation

The following preparation steps aim to create training and validation sets that can be used for machine learning.
For example, if a hentai tag is rare, we will remove it from the target variable.
The model will not learn how to predict that genre if the data covering it is insufficient.

In [ ]:
def plot_label_frequency(labels):
    style.use("fivethirtyeight")
    plt.figure(figsize=(14, 16))
    sns.barplot(
        y=labels.index.values,
        x=labels,
        order=labels.index
    )
    plt.title("Labels", fontsize=14)
    plt.xlabel("Frequency")
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

In [ ]:
# Get label frequencies in descending order
label_freq = hentais_df["tag"].explode().value_counts()
label_freq = label_freq / len(hentais_df["tag"])
plot_label_frequency(label_freq.head(70))

## Remove infrequent labels

We will consider as a rare label every label that is covered by less than 5% in our dataset.
We will assume that rare labels are very hard to predict due to lack of sufficient data.
The model that we will train later will not focus on predicting these labels.
So, we need to make some transformation in the label column (tag) where we ignore infrequent labels by hiding them.

In [ ]:
LABEL_THRESHHOLD = 0.01

In [ ]:
rare_labels_df = label_freq[label_freq < LABEL_THRESHHOLD]
rare_labels_df

Remove the rare tags

In [ ]:
hentais_df = hentais_df.assign(
    tag=hentais_df["tag"].apply(
        lambda x: [l for l in x if l not in rare_labels_df]
    )
)
hentais_df

In [ ]:
plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
hentais_df["num_favorites"].plot.hist(
    bins=100,
    range=(0, 6000),
    title="Favorites"
)
plt.subplot(1, 2, 2)
hentais_df["num_pages"].plot.hist(
    bins=100,
    range=(0, 300),
    title="Pages"
)
plt.show()

In [ ]:
print("Most frequent pages numbers: ", (hentais_df["num_pages"]
                                        .value_counts()
                                        .idxmax()))

Assuming that each unit contains sufficient context, that hentai of more than 30 pages are generally relatively consistent genre antologies over the time window.

In [ ]:
IMG_TIMESTEP = 20 # Number of pages to build a contextual classification of hentai tags

In [ ]:
hentais_df = hentais_df.assign(
    filename=hentais_df.apply(
        lambda x: [
            x["filename"][i:i + IMG_TIMESTEP]
            for i in range(0, len(x["filename"]), IMG_TIMESTEP)
        ],
        axis=1
    )
)
hentais_df = hentais_df.explode("filename")
hentais_df

## Image examples

Let's display some examples of training images.

In [ ]:
hentai_sample_df = (
    hentais_df
    .explode("filename")
    .sample(10)
    .reset_index()
)

plt.figure(figsize=(14, 8))
for i, row in enumerate(hentai_sample_df.iterrows()):
    plt.subplot(2, 5, int(i) + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(Image.open(str(row[1]["filename"])))
    plt.title(str(row[1]["id"]))
plt.show()

## Data preparation

Convert variable size list to Ragged Tensor

In [ ]:
X = tf.ragged.constant(hentais_df["filename"].to_numpy())
y = tf.ragged.constant(hentais_df["tag"].to_numpy())

The original targets are lists of strings that can be easily understood by humans.
But, if we want to build and train a neural network we need to create binary labels (multi-hot encoding).
This is critical for multi-label classification.

In order to binarize our labels, we will be using keras's category encoding layer.

In [ ]:
# Build a index of the feature values and encode the integer into one-hot vector
label_indexer = keras.layers.StringLookup()
label_indexer.adapt(y)

# Encode interger indices to a one-hot vector
label_category_encoder = keras.layers.CategoryEncoding(output_mode="binary")
label_category_encoder.adapt(label_indexer(y))

# Build the category encoding model
tokenizer = keras.Sequential([
    label_indexer,
    label_category_encoder
])

# Analyze the number of labels in the dataset for
# the output layer of our subsequent model
labels_df = pd.Series(tokenizer.get_vocabulary())
labels_df

Transform the targets of the training and test sets.

In [ ]:
y = tokenizer(y)

## Input pipeline

If you are familiar with keras.preprocessing you may know the image data iterators (E.g. ImageDataGenerator, DirectoryIterator).
These iterators are convenient for multi-class classfication where the image directory contains one subdirectory for each class.
But, in the case of multi-label classification, having an image directory that respects this structure is not possible because one observation can belong to multiple classes at the same time.

That is where the tf.data API has the upper hand.
- It is faster.
- It provides fine-grained control.
- It is well integrated with the rest of TensorFlow.

We first need to write some function to parse image files and generate a tensor representing the features and a tensor representing the labels.
- In this function we can resize the image to adapt to the input expected by the model.
- We can also normalize the pixel values to be between 0 and 1. This is a common practice that helps speed up the convergence of training.

If we consider every pixel as a feature, we would like these features to have a similar range so that the gradients don't go out of control and that we only need one global learning rate multiplier.

In [ ]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

To train a model on our dataset we want the data to be:

- Well shuffled
- Batched
- Batches to be available as soon as possible.

These features can be easily added using the tf.data.Dataset abstraction.

In [ ]:
BATCH_SIZE = 32 # Big enough to measure an F1-score
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations
SUFFLE_SEED = 44 # Create reproductible train/test dataset

AUTOTUNE will adapt the preprocessing and prefetching workload to model training and batch consumption.
The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step.
AUTOTUNE will prompt the tf.data runtime to tune the value dynamically at runtime and reduce GPU and CPU idle time.

In [ ]:
# Calculate datasets lenght
dataset_len = len(hentais_df)
val_len = math.floor(dataset_len * 0.2)
train_len = dataset_len - val_len

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y))
# Shuffle before train/test split
dataset = dataset.shuffle(
    buffer_size=SHUFFLE_BUFFER_SIZE,
    seed=SUFFLE_SEED
)

Analyze the number of labels in the dataset for the output layer of our subsequent model.

In [ ]:
nlabels = tokenizer.vocabulary_size()
print("Number of labels in the dataset : {}".format(nlabels))

Transform the targets of the training and test shape.

In [ ]:
def read_image(example: tf.Tensor):
    # Read an image from a file
    image_string = tf.io.read_file(example)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_image(
        image_string,
        channels=CHANNELS,
        expand_animations=False
    )
    # Resize it to fixed shape
    image_resized = tf.image.resize(
        image_decoded,
        [IMG_SIZE, IMG_SIZE]
    )
    return image_resized


def read_image_set(example: tf.Tensor, label: tf.Tensor):
    example = tf.map_fn(
        fn=read_image,
        elems=example,
        fn_output_signature=tf.float32
    )
    return example, label

In [ ]:
# Read and preprocess images in parallel
dataset = dataset.map(read_image_set, num_parallel_calls=tf.data.AUTOTUNE)
# Batch the data for multiple steps
dataset = dataset.padded_batch(
    BATCH_SIZE,
    padded_shapes=(
        [IMG_TIMESTEP, IMG_SIZE, IMG_SIZE, CHANNELS],
        [nlabels]
    )
)

## Train / val split

Splitting the modeling data into training and validation is common in machine learning practice.
We will be allocating 80% of the images for training and 20% for validation.
Usually, we keep a final test set to communicate performance results but we will not really need it in this notebook.

In [ ]:
# Sample test dataset
val_ds = dataset.take(val_len)
# Take the rest of the dataset for training
train_ds = dataset.skip(val_len)
# Fetch batches in the background while the model is training
train_ds = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

## Establish a baseline

Following development best practices, you should establish a baseline. The simplest baseline is predicting the most average classes.

In [ ]:
print("Average class prediction baseline: {0:.0%}".format(np.mean(y)))

## Transfert Learning

The feature extractor accepts images of shape (224, 224, 3) and returns a 1280-length vector for each image.

We should freeze the variables in the feature extractor layer, so that the training only modifies the new classification layers.
Usually, it is a good practice when working with datasets that are very small compared to the orginal dataset the feature extractor was trained on.

In [ ]:
feature_extractor = keras.applications.InceptionV3(
    input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS),
    include_top=False,
    pooling="max"
)
feature_extractor.trainable = False
feature_extractor.summary()

# Data Augmentation

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS)),
    keras.layers.RandomRotation(
        factor=0.4,
        fill_mode="wrap"
    ),
    keras.layers.RandomTranslation(
        height_factor=0.2,
        width_factor=0.2,
        fill_mode="wrap"
    ),
    keras.layers.RandomContrast(factor=0.2),
    keras.layers.Rescaling(1. / 255)
])
data_augmentation.summary()

## Main Model

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(IMG_TIMESTEP, IMG_SIZE, IMG_SIZE, CHANNELS)),
    keras.layers.TimeDistributed(data_augmentation),
    keras.layers.TimeDistributed(feature_extractor),
    keras.layers.GRU(64),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(nlabels, activation="sigmoid")
])
model.summary()

## Model training and evaluation

In [ ]:
@tf.function
def macro_soft_f1(y: tf.Tensor, y_hat: tf.Tensor):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2 * tp + fn + fp + 1e-16)
    # Reduce 1 - soft-f1 in order to increase soft-f1
    cost = 1 - soft_f1
    # Average on all labels
    macro_cost = tf.reduce_mean(cost)
    return macro_cost

In [ ]:
@tf.function
def macro_f1(y: tf.Tensor, y_hat: tf.Tensor, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2 * tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 1

Compile the model to configure the training process.

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=LR),
  loss=macro_soft_f1,
  metrics=[macro_f1]
)

Now, we pass the training dataset of (features, labels) to fit the model and indicate a seperate dataset for validation.
The performance on the validation dataset will be measured after each epoch.

In [ ]:
output_dir = Path("job")
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
savedmodel_dir = output_dir / "export" / "savedmodel"
model_export_path = savedmodel_dir / timestamp
checkpoint_path = output_dir / "checkpoints"
tensorboard_path = output_dir / "tensorboard"

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3),
        keras.callbacks.TensorBoard(
            str(tensorboard_path),
            histogram_freq=1
        ),
        keras.callbacks.ModelCheckpoint(
            checkpoint_path,
            save_weights_only=True,
            save_best_only=True,
            verbose=1
        ),
    ]
)

In [ ]:
plt.figure(figsize=(14, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history["macro_f1"])
plt.plot(history.history["val_macro_f1"])
plt.title("Macro F1")
plt.xlabel("epoch")
plt.legend(["train", "validation"])

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss")
plt.xlabel("epoch")
plt.legend(["train", "validation"])
plt.show()

In [ ]:
model.save(model_export_path)

## Show predictions

We can try and see what the predictions will look like when using our model on pages of some known hentais.
The following function simplifies the process of preparing images data, generating the prediction from the model and visualizing it.

In [ ]:
model = keras.models.load_model(
    model_export_path,
    custom_objects={
        "macro_f1": macro_f1,
        "macro_soft_f1": macro_soft_f1
    }
)

In [ ]:
def decode_image(path: str):
    img = keras.utils.load_img(
        path,
        target_size=(IMG_SIZE, IMG_SIZE, CHANNELS)
    )
    img = keras.utils.img_to_array(img)
    img = img / 255
    return img


def plot_prediction(hentai: pd.Series):
    # Get hentai info
    img_paths = np.arange(1, 11)
    img_paths = np.array([
        str(Path(DATA_PATH) / "preprocessed" / str(hentai["id"]) / "{}.jpg".format(x))
        for x in img_paths
    ])

    # Read and prepare image
    imgs = [
        decode_image(img)
        for img in img_paths[:IMG_TIMESTEP]
    ]
    imgs = np.expand_dims(imgs, axis=0)

    # Generate prediction
    prediction = model.predict(imgs)
    prediction = pd.Series(prediction[0])
    prediction = pd.concat([
        labels_df.rename("label"),
        prediction.rename("prediction")
    ], axis=1)
    prediction = prediction[prediction["prediction"] > 0.5]
    prediction = prediction["label"]

    # Display image with prediction
    style.use("default")
    plt.figure(figsize=(8, 4))
    plt.imshow(Image.open(img_paths[0]))
    plt.title(
        "{}\n\nTag\n{}\n\nPrediction\n{}\n".format(
            hentai["title"],
            hentai["tag"],
            list(prediction)
        ),
         fontsize=9
    )
    plt.show()

In [ ]:
sampled_hentais_df = hentais_df[
    hentais_df["title"].isin([
        "My Little Brother",
        "Maotoko Rental Service",
        "Maeoki wa Iranu Warawa to Asobe",
        "Jigoku e no Katamichi 1 Credit",
        "Makuu GB Tsuushin 3"
    ])
]

for _, sampled_hentai in sampled_hentais_df.iterrows():
    plot_prediction(sampled_hentai)